<span style="font-size: 14pt">MIPT, Applied ML, Autumn 2018</span>

<span style="font-size: 16pt"> HW #4: Approximate RL homework

<span style="color:red; font-size: 14pt;"> Дедлайн 19.11.2018 23:59 </span>

<span style="color:blue; font-size: 12pt">Valentin Malykh </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">val@maly.hk</span>


**Оформление дз**: 
- Выполненное задание требуется отправлять через <a href='https://goo.gl/forms/XPSIbwp7wPxB4SsI3'>форму </a>

- Выполненное дз прикрепляйте в формате файла ``<фамилия>_<группа>_task<номер>.ipynb``, например: ``ivanov_594_task4.ipynb`` 

**Вопросы**:
- Вопросы присылайте в канал в телеграмме ``[Fall 2018]ML Seminars``

--------
- **PS1**: Будьте внимательны при заполнении формы, когда отправляете ДЗ 


<h1 align="center">Практическое задание (100%)</h1>
Описание находится по ссылке: https://gist.github.com/madrugado/1262c3077bf7d8ac8166e4350f0f67e2

# Попробуем сделать как на семинаре

## 1. Загрузка environment-a

In [1]:
import gym
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
env = gym.make("LunarLander-v2")
env.reset()

state_dim = env.observation_space.shape
n_actions = env.action_space.n

In [3]:
#plt.imshow(env.render("rgb_array"))

In [4]:
print(state_dim, n_actions)

(8,) 4


# Experience replay

In [5]:
import random
import collections

class ExperienceBuffer():
    def __init__(self, buffer_size=10000):
        self.buffer = collections.deque(maxlen=buffer_size)
        self.buffer_size = buffer_size
    
    def add(self, experience):
        # experience == [s, a, r, s', is_end]
        self.buffer.append(experience)
            
    def sample(self, size=1):
        # [s, a, r, s', is_end]
        return random.sample(self.buffer, size)

## Создадим DQN-сеть

First step is initializing input variables

**Tensorflow 1.11.0 version required.**

In [6]:
import tensorflow.contrib.layers as tflayers  # Let's make TF simple again

In [7]:
#create input variables. We'll support multiple states at once
current_states = tf.placeholder(dtype=tf.float32,shape=(None,)+state_dim)
actions = tf.placeholder(tf.int32,shape=[None])
rewards = tf.placeholder(tf.float32,shape=[None])
next_states = tf.placeholder(tf.float32, shape=(None,)+state_dim)
is_end = tf.placeholder(tf.bool,shape=[None])

In [8]:
def network(l_states, scope=None, reuse=False):
    assert l_states.get_shape().as_list() == list((None,)+state_dim)
    with tf.variable_scope(scope or "network") as scope:
        if reuse:
            scope.reuse_variables()
        dense_1 = tflayers.fully_connected(l_states, 200, activation_fn=tf.nn.relu)
        dense_2 = tflayers.fully_connected(dense_1, 40, activation_fn=tf.nn.relu)
        l_qvalues = tflayers.fully_connected(dense_2, n_actions, activation_fn=None)
        return l_qvalues

#### Predicting Q-values for `current_states`

In [9]:
#get q-values for ALL actions in current_states
predicted_qvalues = network(current_states)

In [10]:
#select q-values for chosen actions
batch_size = tf.shape(predicted_qvalues)[0]
gather_ids = tf.stack([tf.range(batch_size), actions], axis=1)
predicted_qvalues_for_actions = tf.gather_nd(predicted_qvalues, gather_ids)

In [11]:
predicted_qvalues_for_actions.shape

TensorShape([Dimension(None)])

#### Loss function and `update`
Here we write a function similar to `agent.update`.

In [12]:
predicted_next_qvalues = network(next_states, reuse=True)
gamma = 0.99
target_qvalues_for_actions = tf.add(rewards, tf.multiply(gamma, tf.reduce_max(predicted_next_qvalues, axis=1)))
target_qvalues_for_actions = tf.where(
    is_end, 
    tf.zeros_like(target_qvalues_for_actions),
    target_qvalues_for_actions)

In [13]:
#mean squared error loss function
loss = tf.losses.mean_squared_error(target_qvalues_for_actions, predicted_qvalues_for_actions)

In [14]:
#network updates. Note the small learning rate (for stability)
#Training function that resembles agent.update(state,action,reward,next_state) 
#with 1 more argument meaning is_end
train_step = tf.train.AdamOptimizer(1e-4).minimize(
    loss, var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="network"))

## Обучим

In [15]:
# Tensorflow feature - session
sess = tf.InteractiveSession()

In [16]:
# Tensorflow feature 2 - variables initializer
sess.run(tf.global_variables_initializer())

In [17]:
# You can check all your valiables by:
# [v.name for v in tf.trainable_variables()]
# they should all starts with "network"

In [18]:
def generate_session(exp_buffer, t_max=1000, train_freq=8, batch_size=16, print_result=False):
    """play env with approximate q-learning agent and train it at the same time"""
    
    total_reward = 0
    s = env.reset()
    total_loss = 0
    
    for t in range(t_max):
        # get action q-values from the network
        q_values = sess.run(
            predicted_qvalues, 
            feed_dict={current_states: np.array([s])}
        )[0]
        
        a = np.random.choice(
            [np.argmax(q_values), np.random.randint(n_actions)], p=[1 - epsilon, epsilon]
        )
        
        new_s, r, done, info = env.step(a)
        exp_buffer.add([s, a, r, new_s, done])
        
        if t % train_freq == 0 and len(exp_buffer.buffer) >= batch_size:
            # train agent one step. Note that we use one-element arrays instead of scalars 
            # because that's what function accepts.
            exps = exp_buffer.sample(size=batch_size)
            s_s = np.array([exp[0] for exp in exps])
            a_s = np.array([exp[1] for exp in exps])
            r_s = np.array([exp[2] for exp in exps])
            ns_s = np.array([exp[3] for exp in exps])
            ends = np.array([exp[4] for exp in exps])
            curr_loss, _ = sess.run(
                [loss, train_step], 
                feed_dict={
                    current_states: s_s,
                    actions: a_s,
                    rewards: r_s,
                    next_states: ns_s,
                    is_end: ends,
                })
            total_loss += curr_loss

        total_reward += r
        
        s = new_s
        if done: 
            break
    if print_result:        
        if r >= 0:
            print("Landed successfully")
        if r < 0:
            print("Failed")
            
    return total_reward, total_loss/float(t), t

In [19]:
inial_epsilon = epsilon = 0.2
final_epsilon = 0.01
n_epochs = 20
exp_buffer = ExperienceBuffer(buffer_size=5000)

In [20]:
# fine-tune model from epsilon=0.1
epsilon = 0.1
env = gym.make("LunarLander-v2")
env.reset()

array([ 0.00529394,  1.4131014 ,  0.536199  ,  0.09693115, -0.00612753,
       -0.1214569 ,  0.        ,  0.        ], dtype=float32)

In [21]:
from tqdm import tqdm_notebook
n_pos_epochs = 0
for i in tqdm_notebook(range(n_epochs)):
    sessions = [generate_session(exp_buffer,
                                 t_max=1000,
                                 train_freq=8,
                                 batch_size=16,
                                 print_result=False)
                for _ in range(300)] #generate new sessions
    session_rewards, session_loss, session_steps = map(np.array, zip(*sessions))
    
    #epsilon -= (inial_epsilon - final_epsilon) / float(n_epochs)
    if epsilon > final_epsilon:
        epsilon = epsilon * 0.95
    
    print("mean reward = {:.3f}\tepsilon = {:.3f}\tloss = {:.3f}\tsteps = {:.3f}".format(
        np.mean(session_rewards), epsilon, np.mean(session_loss), np.mean(session_steps)))
    
    # If we won three times in a row, stop learning
    if np.mean(session_rewards) > 0:
        n_pos_epochs += 1
    else:
        n_pos_epochs = 0
    if n_pos_epochs >= 3:
        break
        
    assert epsilon!=0, "Please explore environment"

mean reward = -477.392	epsilon = 0.095	loss = 7.601	steps = 68.403


KeyboardInterrupt: 

In [ ]:
epsilon = 0.01

## Протестим

In [22]:
#record sessions
import gym.wrappers
env = gym.make("LunarLander-v2")
#env.reset()
env = gym.wrappers.Monitor(env,directory="videos",force=True)
sessions = [generate_session(exp_buffer, t_max=1000, train_freq=10000, batch_size=1, print_result=True) for _ in range(10)]
env.close()
#unwrap 
env = env.env.env

Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed


Садится успешно 5 раз из 10. Неплохо для такой тупой архитектуры

In [ ]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices